# Práctica 4: Procesamieto de audio

### **Participantes:**
- Gerardo León Quintana
- Susana Suárez Mendoza

In [2]:
from scipy.fft import fft
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

## Ejercicio 1:

Construir un identificador de notas musicales. Es decir; en su versión más sencilla (y suficiente) la entrada es un sonido con una sola nota musical y debe identificar cuál es. Por simplicidad elija un único instrumento para la identificación. 

In [3]:
NOTES_FREQS = {
    "C4": 261.63, "C#4/Db4": 277.18, "D4": 293.66, "D#4/Eb4": 311.13, "E4": 329.63, 
    "F4": 349.23, "F#4/Gb4": 369.99, "G4": 392.00, "G#4/Ab4": 415.30, "A4": 440.00, 
    "A#4/Bb4": 466.16, "B4": 493.88, "C5": 523.25
}

In [4]:
def closest_note(freq):
    """Identifica la nota más cercana a una frecuencia dada."""
    note = min(NOTES_FREQS.keys(), key=lambda note: abs(NOTES_FREQS[note] - freq))
    return note

In [5]:

def identify_notes_from_wav(file, window_size=2048, hop_size=1024):
    # Cargar el archivo WAV usando librosa
    sample_rate, data = wavfile.read(file)

    # Número total de ventanas
    num_windows = (len(data) - window_size) // hop_size + 1
    
    notes_identified = []
    
    for i in range(num_windows):
        start = i * hop_size
        end = start + window_size
        
        # Tomar un segmento de la señal
        segment = data[start:end]

        # Aplicar la FFT al segmento
        N = len(segment)
        yf = fft(segment)
        xf = np.fft.fftfreq(N, 1 / sample_rate)

        # Obtener la frecuencia dominante
        idx = np.argmax(np.abs(yf[:N // 2]))  # Consideramos solo la primera mitad de la FFT
        dominant_freq = abs(xf[idx])

        # Encontrar la nota más cercana a la frecuencia dominante
        note = closest_note(dominant_freq)
        notes_identified.append(note)

        # Graficar el espectro de frecuencias del segmento
        """plt.figure(figsize=(10, 4))
        plt.plot(xf[:N // 2], np.abs(yf[:N // 2]))
        plt.title(f'Espectro de frecuencias de la ventana {i+1}')
        plt.xlabel('Frecuencia [Hz]')
        plt.ylabel('Amplitud')
        plt.xlim(0, 600)  # Limitar a 600 Hz para mejor visualización
        plt.grid()"""
        #plt.show()

    # Mostrar resultados
    for note in notes_identified:
        print(f"Nota identificada: {note}")

# Ejemplo de uso


In [8]:
identify_notes_from_wav('./audios/do_piano.wav')  

Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: C4
Nota identificada: E4
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota identificada: G4
Nota identificada: C5
Nota identificada: C5
Nota identificada: C5
Nota ident

## Ejercicio 2:

Construir una pequeña aplicación que permita operar con diferentes filtros (con un selector) y trabajar con varios umbrales. (uno para los filtros pasa-bajo y pasa-alto y dos para los filtros pasa-banda y rechaza-banda). Demuestre su funcionalidad con señales ruidosas. Muestre en cada filtrado la señal original y filtrada en el dominio temporal y en el dominio de la frecuencia.